# 1. Business Understanding

## 1.1 Project Background

Ocular diseases such as Diabetic Retinopathy (DR), Glaucoma, and Cataracts represent a significant and growing global health burden. These conditions are leading causes of preventable blindness worldwide. The key to preventing vision loss is early and accurate detection.

Currently, diagnosis relies on a manual examination of retinal fundus images by highly trained ophthalmologists. This process, while effective, faces several critical challenges:  

- **Scalability & Accessibility:** There is a global shortage of ophthalmologists, particularly in remote and underserved regions. This creates a severe bottleneck, leading to long wait times for screenings and delayed diagnoses.  
- **Time-Consuming & Repetitive:** Manual screening is a time-intensive task that consumes a significant portion of a specialist's day, much of which is spent reviewing normal, healthy eye scans.  
- **Human Factor:** The diagnostic process is subject to human fatigue and inter-observer variability, which can lead to inconsistent or missed findings.  

The convergence of deep learning, particularly in computer vision, and the increased availability of digital fundus imagery presents a transformative opportunity to address these challenges.

---

## 1.2 Problem Statement

The current manual screening process for ocular diseases is inefficient, unscalable, and inaccessible to large parts of the population, leading to preventable vision loss due to late detection.  

Healthcare providers require a tool that can automate the initial screening process. This tool must analyze a retinal fundus image and accurately identify the presence of multiple potential pathologies simultaneously.  

This project addresses the need for an assistive tool by tackling this as a multi-label classification problem, where a single image can be flagged for one or more diseases.

---

## 1.3 Project Objectives

The primary objective of this project is to develop and deploy a proof-of-concept Decision Support System (DSS) for ophthalmologists and general practitioners. This system will leverage a deep learning model to serve as an automated, first-pass screening tool.

The specific, measurable objectives are:

- **To Develop a Multi-Label Model:** To build, train, and validate a Convolutional Neural Network (CNN) model that can accurately detect the presence of eight distinct ocular pathologies from a single fundus image: Normal, Diabetes, Glaucoma, Cataract, Age-related Macular Degeneration (AMD), Hypertension, Myopia, and Other abnormalities.  
- **To Prioritize Triage:** The model's primary function is not to replace a doctor, but to act as a triage assistant. It will help clinicians prioritize patient caseloads by automatically flagging images with high-risk indicators for immediate review.  
- **To Enhance Efficiency:** By automating the screening of healthy/normal scans, the system will aim to reduce the manual review burden on specialists, allowing them to dedicate their time to complex diagnoses and treatment.  
- **To Deploy an Accessible Tool:** To deploy the final trained model as an interactive web application, demonstrating a tangible, user-friendly prototype that can be accessed by healthcare professionals for evaluation.

---

## 1.4 Business Success Criteria

This academic project will be evaluated on both its technical performance and its practical utility.

- **Primary Technical Metric:** Due to the severe class imbalance inherent in medical datasets (where 'Normal' cases far outnumber rare diseases), standard accuracy is a poor metric. Therefore, the model's success will be primarily measured by the mean Area Under the Receiver Operating Characteristic Curve (AUC-ROC) across all 8 classes.  

  **Target:** A mean AUC-ROC score of \( \ge 0.90 \) on the hold-out test set.  

  **Rationale:** This metric effectively measures the model's ability to distinguish between positive and negative cases for each disease, even for the rare classes.  

- **Secondary Technical Metric:** The per-class F1-Score, Precision, and Recall will be reported to provide a transparent view of the model's performance on common vs. rare conditions.  

- **Deployment & Utility Metric:** The project will be considered a success upon the successful deployment of a functional web-based application. This application must allow a user to upload a fundus image and receive a clear, human-readable probabilistic output for all 8 disease categories, successfully demonstrating the project's value as a proof-of-concept.
